In [8]:
from dataclasses import dataclass, field
from typing import Optional
from datetime import datetime
import yfinance as yf

## Engine : Yield curve interpolation for cash flow discounting
## 

In [ ]:
(datetime.today()-datetime.strptime('2025-08-08'), '%Y-%m-%d').days/365

TypeError: strptime() takes exactly 2 arguments (1 given)

In [ ]:
frequency_mapping_dict = {
    'Annually': 1,
    'Semi-Annually': 2,
    'Quarterly': 4
}

@dataclass 
class Stock:
    Ticker: str

    def price(self):
        return yf.Ticker(self.Ticker).fast_info['lastPrice']

@dataclass
class Interest_Rates:
    Reference_Benchmark: str 
    Maturity: str
    Frequency: str

    Num_Frequency: int = field(init=False)
    T: int = field(init=False)

    def __post_init__(self):
        try:
            Num_Frequency = frequency_mapping_dict[self.Frequency]
        except KeyError:
            raise ValueError(f'Incorrect Frequency: {self.Frequency} should be one of {list(frequency_mapping_dict.keys())}')

        try:
            T = (datetime.strptime(self.Maturity, '%Y-%m-%d')-datetime.today).days()/365
        except ValueError:
            raise ValueError(f'Incorrect date : {self.Maturity}. Please use YYYY-MM-DD format')


    

@dataclass
class Convertible:
    Face_Value: float
    Coupon: float
    Coupon_Frequency : str
    Conversion_Ratio : float
    Conversion_Strike : float 

    First_Coupon_Date : str
    Maturity: str
    Redemption : float
    Underlying_stock_ticker : str
    Day_Count_Convention : Optional[str] = 'ACT/ACT'
    Bond_Currency : Optional[str] = 'EUR'
    Stock_Currency : Optional[str] = 'EUR'
    AI_on_Conversion: Optional[bool] = True

    def conversion_price(self):
        return self.Face_Value/self.Conversion_Ratio
    
    def parity(self):
        #Equivalent value in shares if converted right now
        return self.

@dataclass
class Option:
    Underlying: str
    K : float
    T : str  
    r = yf.Ticker('e')
